# Working with the Transformers Library for Different Natural Language Processing Tasks


Within this notebook we will use the the **Transformers Library** by [Hugging Face](https://huggingface.co/). The library consists of thousands of pre-trained models many of which are trained on huge datasets for thousands of GPU hours. You can use them either directly for inference (as we will do in this lab session) or fine-tune them for your specific applications. Using pre-trained models allows you to reduce your compute costs and carbon footprint and save time and resources required to develop a model from scratch.

For creating user interfaces we will use Gradio which you already know from the previous session on using ML APIs.

In [ ]:
!pip install gradio
!pip install transformers[sentencepiece]
!pip install transformers
!pip install torch

The HuggingFace [ModelHub](https://huggingface.co/models) consists of various pre-trained models for different tasks which can be downloaded and used easily using the Transformers Library. 

##  Transformers for Translation Tasks

The easiest way to use a pre-trained model for inference is the **pipeline**. The pipeline can be used out-of-the box for many tasks across modalities (e.g., text, images, etc.). In this lab session we will look into a subset including translation and text classification.

<table>
  <tr>
    <th>Task</th>
    <th>Description</th>
    <th>Pipeline identifier</th>
  </tr>
  <tr>
    <td>Translation</td>
    <td>translate text from one language into another</td>
    <td>pipeline(task=“translation”)</td>
  </tr>
  <tr>
    <td>Text classification</td>
    <td>assign a label to a given sequence of text</td>
    <td>pipeline(task=“sentiment-analysis”)</td>
  </tr>
  <tr>
    <td>Text generation</td>
    <td>generate text that follows a given prompt</td>
    <td>pipeline(task=“text-generation”)</td>
  </tr>
  <tr>
    <td>Image Classification</td>
    <td>assign a label to an image</td>
    <td>pipeline(task=“image-classification”)</td>
  </tr>
</table>

For a comprehensive overview you can click [here](https://huggingface.co/docs/transformers/main/en/quicktour#pipeline).


As a first example we will explore the pipeline for translating text from one language to another one (use `pipeline("translation_xx_to_yy")`) For example, to translate from English to German you can use `pipeline("translation_en_to_de")`. 

The pipeline downloads and caches a default pretrained model. You can then use it on your target text, e.g., `response = translator(text)`.  

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")

input_text = "Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. "
translated_text = translator(input_text)
print(translated_text)

Rather than using a default model, we will now use a **specific model** for the translation task and pass the model, which should be used for the translation to the pipeline as parameter. 

We will use `mdl_name = "Helsinki-NLP/opus-mt-en-de"`. You can find details concerning the model on its [model card](https://huggingface.co/Helsinki-NLP/opus-mt-en-de) in the ModelHub.

In [ ]:
mdl_name = "Helsinki-NLP/opus-mt-en-de"
translator = pipeline("translation", model=mdl_name)

input_text = "Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. "
translated_text = translator(input_text)
print(translated_text)

The Gradio library which we already used during the last session allows to load almost every model on HuggingFace. For this, we use the class level method `load` (i.e., `gradio.Interface.load(name, ···)`). The input and output components of the user interface are then automatically loaded from the repository. 

In [ ]:
import gradio as grad

grad.Interface.load("huggingface/Helsinki-NLP/opus-mt-en-de").launch()

Alternatively, we can also load the user interface from a pipeline. For this, we use the class level method `from_pipeline` (i.e., `gradio.Interface.from_pipeline(name, ···)`). The input and output components of the user interface are then automatically determined from the pipeline. 

In [ ]:
import gradio as grad

grad.Interface.from_pipeline(translator).launch()

You can configure title and description of the user interface and also provide examples

In [ ]:
import gradio as grad

title = "Translation"
description = "Translates text from English to German ... "
examples = [["Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch."]]

grad.Interface.from_pipeline(translator, title=title, description=description, examples=examples).launch()

##  Transformers for Text Classification 

In a next step we use the Transformers library for a text classification task (using `pipeline("sentiment-analysis")`). As model we use a specific model (i.e., 
`mdl_name = "siebert/sentiment-roberta-large-english"`), which is a fine-tuned checkpoint of a RoBERTa large model. 

In [ ]:
from transformers import pipeline
import gradio as grad

mdl_name = "siebert/sentiment-roberta-large-english"
roberta_pipe = pipeline("sentiment-analysis", model=mdl_name)

title = "Sentiment Analysis"
description = "assigns a label to a given sequence of text ... "
examples = [["I am super happy today"], ["The weather is awful"],]

grad.Interface.from_pipeline(roberta_pipe, title=title, description=description, examples=examples).launch(debug=True)

##  Transformers for Text Generation

In a next step we use the Transformers library for a text generation task (using `pipeline("text-generation")`).

As model we use a specific model (i.e., `model = "GPT2"`). We will configure the `generator` by limiting the length of the output (using `max_length`) and by specifying the number of sequences that should be generated (using `num_return_sequences`).

In [ ]:
from transformers import pipeline, set_seed
import gradio as grad

def generate(text):
  generator = pipeline('text-generation', model='gpt2')
  set_seed(42)
  generated_text = generator(text, max_length=30, num_return_sequences=1)
  return generated_text[0]["generated_text"]

examples = [["Once upon a time"]]
txt=grad.Textbox(lines=1, label="Your input", placeholder="Once upon")
grad.Interface(generate, 
               inputs=txt, 
               outputs="text",
               examples=examples).launch()

##  Transformers for Text Summarization

We will now try a text summarization task. As pre-trained model we will rely on `deep-learning-analytics/wikihow-t5-small`. For more info on the model you can visit the [ModelHub](https://huggingface.co/deep-learning-analytics/wikihow-t5-small).

**Create your own pipeline**

Since the default pipelines only support limited scenarios we now look into how to create our own pipeline.

In a text summarization task we have a piece of text as an input and would like to return a summary of this text. However, we cannot feed this input text directly into the language model. The language model expects a tensor with the IDs referring to the token indices.  For this we use a tokenizer that can help with (1) splitting the text into words and sub-words and (2) mapping each token to an integer.

On a more technical level we use a `Tokenizer`, more specifically an `AutoTokenizer`, which we can instantiate from an existing file on the Hugging Face Hub (i.e., `tokenizer = AutoTokenizer.from_pretrained(model_name)`). 

A `Tokenizer` can then be used to encode the input text (here `text_input_sequence`) and provide the encoding required by the language model (e.g., `encoded_text = tokenizer.encode(text_input_sequence, return_tensors="pt")`; `pt` stands for Pytorch). 


In [ ]:
from transformers import AutoTokenizer
import gradio as grad

model_name = "deep-learning-analytics/wikihow-t5-small" 
tokenizer = AutoTokenizer.from_pretrained(model_name)

text_input_sequence = "Hello, my name is Barbara"
#Longer text snippet for experimentation - taken from https://huggingface.co/deep-learning-analytics/wikihow-t5-small
#text_input_sequence = """Lack of fluids can lead to dry mouth, which is a leading cause of bad breath. Water can also dilute any chemicals in your mouth or gut that are causing bad breath., Studies show that eating 6 ounces of yogurt a day reduces the level of odor-causing compounds in the mouth. In particular, look for yogurt containing the active bacteria Streptococcus thermophilus or Lactobacillus bulgaricus., The abrasive nature of fibrous fruits and vegetables helps to clean teeth, while the vitamins, antioxidants, and acids they contain improve dental health.Foods that can be particularly helpful include:Apples — Apples contain vitamin C, which is necessary for health gums, as well as malic acid, which helps to whiten teeth.Carrots — Carrots are rich in vitamin A, which strengthens tooth enamel.Celery — Chewing celery produces a lot of saliva, which helps to neutralize bacteria that cause bad breath.Pineapples — Pineapples contain bromelain, an enzyme that cleans the mouth., These teas have been shown to kill the bacteria that cause bad breath and plaque., An upset stomach can lead to burping, which contributes to bad breath. Don’t eat foods that upset your stomach, or if you do, use antacids. If you are lactose intolerant, try lactase tablets., They can all cause bad breath. If you do eat them, bring sugar-free gum or a toothbrush and toothpaste to freshen your mouth afterwards., Diets low in carbohydrates lead to ketosis — a state in which the body burns primarily fat instead of carbohydrates for energy. This may be good for your waistline, but it also produces chemicals called ketones, which contribute to bad breath.To stop the problem, you must change your diet. Or, you can combat the smell in one of these ways:Drink lots of water to dilute the ketones.Chew sugarless gum or suck on sugarless mints.Chew mint leaves."""
encoded_text = tokenizer.encode(text_input_sequence,return_tensors="pt") #pt for pytorch and tf for tensorflow
print(encoded_text)

#For testing purposes we use the method convert_ids_to_tokens to convert the ids into tokens
tokens = tokenizer.convert_ids_to_tokens(encoded_text[0])
print(tokens)


Now that we have looked into how a `Tokenizer` can be used, we will look into how we can use the language model to generate a summary. 

First we download a pre-trained model the same as we downloaded the tokenizer in the above step and instantiate the model. 

```
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

```
Then we pass the encoded text as a tensor to the language model and use the model to generate the summary (i.e., `summary_ids`). The following parameters are used: 

*  `tkn_text` representing the tensor with the IDs
*  `max_length` allows to restrict the length of the generated token
* `num_beams` number of beams for beam search; default 1 (means no beam search)
* `repetition_penalty` is the parameter for repetition penality; default 1.0 stands for no penalty 
* `length_penalty` is the parameter for the length penalty; values > 0 promote longer sequences; values < 0 shorter sequences
* `early_stopping` defaults to False - When `True`, generation finishes if the EOS token is reached

```
summary_ids = model.generate(
          tkn_text,
          max_length=250, 
          num_beams=5,
          repetition_penalty=2.5, 
          length_penalty=1.0, 
          early_stopping=True
)
```

Finally, we use the `Tokenizer` to decode the result provided by the language model (i.e., the tensor with the IDs) into text, i.e., `tokens = tokenizer.decode(encoded_text[0], skip_special_tokens=True)`. 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as grad

# Define the model repo
model_name = "deep-learning-analytics/wikihow-t5-small" 

# Download model
txt2txt_tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def text2text_summary(paragraph):
    #Remove line breaks
    initial_txt = paragraph.strip().replace("\n","")
    #Encode initial_text as tensor
    tkn_text = txt2txt_tokenizer.encode(initial_txt,return_tensors="pt") 
    #Generate summary using model
    summary_ids = model.generate(
            tkn_text,
            max_length=250, 
            num_beams=5,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
    )
    #print(summary_ids)
    #Decode the list of IDs 
    response = txt2txt_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return response

#Examples
examples = [["Lack of fluids can lead to dry mouth, which is a leading cause of bad breath. Water can also dilute any chemicals in your mouth or gut that are causing bad breath., Studies show that eating 6 ounces of yogurt a day reduces the level of odor-causing compounds in the mouth. In particular, look for yogurt containing the active bacteria Streptococcus thermophilus or Lactobacillus bulgaricus., The abrasive nature of fibrous fruits and vegetables helps to clean teeth, while the vitamins, antioxidants, and acids they contain improve dental health.Foods that can be particularly helpful include:Apples — Apples contain vitamin C, which is necessary for health gums, as well as malic acid, which helps to whiten teeth.Carrots — Carrots are rich in vitamin A, which strengthens tooth enamel.Celery — Chewing celery produces a lot of saliva, which helps to neutralize bacteria that cause bad breath.Pineapples — Pineapples contain bromelain, an enzyme that cleans the mouth., These teas have been shown to kill the bacteria that cause bad breath and plaque., An upset stomach can lead to burping, which contributes to bad breath. Don’t eat foods that upset your stomach, or if you do, use antacids. If you are lactose intolerant, try lactase tablets., They can all cause bad breath. If you do eat them, bring sugar-free gum or a toothbrush and toothpaste to freshen your mouth afterwards., Diets low in carbohydrates lead to ketosis — a state in which the body burns primarily fat instead of carbohydrates for energy. This may be good for your waistline, but it also produces chemicals called ketones, which contribute to bad breath.To stop the problem, you must change your diet. Or, you can combat the smell in one of these ways:Drink lots of water to dilute the ketones.Chew sugarless gum or suck on sugarless mints.Chew mint leaves."]]


#Gradio User Interface
para=grad.Textbox(lines=10, label="Paragraph", placeholder="Add your text here")
out=grad.Textbox(lines=1,label="Summary")
grad.Interface(text2text_summary, 
               inputs=para, 
               outputs=out, 
               examples=examples).launch(debug=True)



## Creating a Chatbot/Dialog Bot

We will now create a chatbot with HuggingFace Transformers. 

First of all, we load a tokenizer and a model instance for a specific variant of DialoGPT (e.g., `microsoft/DialoGPT-large`, `microsoft/DialoGPT-medium` or `microsoft/DialoGPT-small`). The tokenizer will help with encoding and decoding and the model will be used to generate responses. For our purposes we will use the `AutoTokenizer` and the `AutoModelForCausalLM`. 

Then we define a function `converse`. Using the tokenizer, the model, and the chat_history, a response to the user input is generated. In a first step, the user input and an End-of-String (EOS) token are encoded. Then in a second step, these are appended to the chat history. The 1000 most recent tokens (i.e.,  `max_length=1000`) are then used in a third step by DialogGPT for generating responses. Finally, the response is then decoded and transformed into tuples (i.e., pairs of user input and corresponding answer by the bot) as needed by the user interface.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

def converse(user_input, chat_history=[]):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    user_input_ids = tokenizer.encode(user_input,tokenizer.eos_token,return_tensors='pt')
 
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor(chat_history), user_input_ids], dim=-1) 
    
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()
    #print(chat_history)

    response = tokenizer.decode(chat_history[0]).split("<|endoftext|>")
    #print(response) 
   
    # convert to tuples of list
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  
    #print(response) 
   
    return response, chat_history


import gradio as gr
gr.Interface(fn=converse,
             theme="default",
             inputs=[gr.Textbox(placeholder="Let's chat"), "state"],
             outputs=["chatbot", "state"]).launch(debug=False)


Now that we have seen how a chatbot can be created using the Transformers libary, we will use Gradio to automatically create a user interface for two different models (`microsoft/DialoGPT-large` and `facebook/blenderbot-400M-distill`). This requires very little programming skills. At the same time there is less possibility to configure the chatbot.


In [ ]:
import gradio as grad
grad.Interface.load("huggingface/microsoft/DialoGPT-large").launch()

In [ ]:
import gradio as grad
grad.Interface.load("huggingface/facebook/blenderbot-400M-distill").launch()

## Comparing Multiple Models and Combining Multiple ML Tasks


###Comparing Models using Gradio

Gradio can also be used to load several models in parallel to compare them.

In [ ]:
import gradio as grad
from gradio.mix import Parallel, Series

examples = [["Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch."]]

io1 = grad.Interface.load("huggingface/Helsinki-NLP/opus-mt-en-de")
io2 = grad.Interface.load("huggingface/t5-base")
Parallel (io1,io2, examples=examples).launch(debug=True)

### Combining Multiple Models using Gradio

We can also use Gradio to combine multiple models to more complex applications. For example, we can with just a few lines create a simple application to translate German text to English and to summarize it.

In [ ]:
import gradio as grad

examples = [["""Haben Hausarbeiten eine Zukunft?
Dank ChatGPT ist jeder Text im Nu geschrieben. Es ist an der Zeit, über Rolle und Bedeutung von Hausarbeiten an Schulen und Universitäten nachzudenken. Ein Beitrag von Vito Roberto und Roman Schister.
«Schriftliche Arbeiten sind ein wichtiger Bestandteil der Ausbildung, da sie den Schülern und Studenten helfen, ihr Verständnis bestimmter Themen zu vertiefen und ihre Fähigkeiten im Schreiben und Argumentieren zu verbessern. Sodann ermöglichen sie den Lehrern, die Leistungen der Schüler und Studenten zu bewerten und ihnen Feedback zu geben, damit sie ihre Kenntnisse und Fähigkeiten verbessern können.»

Den vorstehenden Absatz hat der Chatbot «ChatGPT» geschrieben, den das US-amerikanische Unternehmen OpenAI letzten November veröffentlicht hat. Das Programm musste lediglich gefragt werden: «Welche Bedeutung haben schriftliche Arbeiten in der Ausbildung?» Das Ergebnis lässt sich sehen: Der produzierte Text enthält für die gestellte Frage wesentliche Punkte, ist kohärent, sprachlich korrekt und könnte auch von einem Menschen verfasst worden sein."""]]

translator = grad.Interface.load("huggingface/Helsinki-NLP/opus-mt-de-en")
summarizer = grad.Interface.load("huggingface/deep-learning-analytics/wikihow-t5-small")
Series(translator, summarizer, inputs=grad.Textbox(lines=10, label="Input Text in German", placeholder="Add your text here"), examples=examples).launch(debug=True)

### Combining Label Detection with Translation (for Google Colab User)


We will now combine the code for label detection via an existing API you already know from the previous session with a translation task.

In [ ]:
# Install the Google Cloud Vision library that allows us to make request to the Google Cloud Vision API
!pip install google-cloud-vision

In [ ]:
# Import the libraries
from google.cloud import vision
import os
import json

In [ ]:
credentials = {
##COPY the content of the JSON file here##   

}

    
json_credentials = json.dumps(credentials) 

with open('My Project-543e6ed386ee.json','w') as outfile:
  outfile.write(json_credentials)

In [ ]:
# Using the GOOGLE_APPLICATION_CREDENTIALS environment variable the location of a credential JSON file can be provided. 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'My Project-543e6ed386ee.json'

In [ ]:
# Instantiate the client (this only works with the credantials correctly set)
client = vision.ImageAnnotatorClient()

In [ ]:
#Import libraries
from PIL import Image as PillowImage
import io

In [ ]:
# Adapted from https://stackoverflow.com/questions/33101935/convert-pil-image-to-byte-array
# Conversion of image into byte array
def image_to_byte_array(filePath) -> bytes:
  #Open File as PIL Image
  pil = PillowImage.open(filePath, "r")   
  # BytesIO is a fake file stored in memory
  imgByteArr = io.BytesIO()
  # image.save expects a file as a argument, passing BytesIO object
  pil.save(imgByteArr, format=pil.format)
  # Turn the BytesIO object back into a bytes object
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

In [ ]:
from transformers import pipeline

def detect_and_translate_text(filePath, translator):
  image = vision.Image()
  image.content = image_to_byte_array(filePath)
  response_text = client.text_detection(image=image)

  text=""
  if response_text.text_annotations:
    text = response_text.text_annotations[0].description
  
  #Translate output of label detection
  mdl_name = "Helsinki-NLP/opus-mt-de-en"
  translator = pipeline("translation", model=mdl_name)
  response = translator(text)
  
  return response

In [ ]:
import gradio as grad
demo = grad.Interface(
    detect_and_translate_text, 
    inputs=grad.Image(type="filepath"),
    outputs = grad.TextArea()
)
demo.launch(debug=True)

# Exercise

Now try for yourself and get your hands dirty.

Modify the example to translate from english to spanish. Search the [ModelHub](https://huggingface.co/models) to find a model which allows translating from english to spanish.

In [ ]:
##Add your code to translate from English to Spanish here including the user interface

from transformers import pipeline
import gradio as grad
mdl_name = "Helsinki-NLP/opus-mt-en-es"
opus_translator = pipeline("translation", model=mdl_name)
grad.Interface.from_pipeline(opus_translator).launch()


Now try to create a user interface for image classifcation. Use the pipeline abstraction with task `image-classification`. As model use `google/mobilenet_v2_1.0_224`.


In [ ]:
##Add your code for image classification here 

from transformers import pipeline
classifier = pipeline("image-classification", model="google/mobilenet_v2_1.0_224")
grad.Interface.from_pipeline(classifier).launch()